# Classification of Hyperspectral Images (HSI) 

This notebook uses [hsi_env](../../environments/hsi_env.yml) conda environment for running. Take a look in [README](../../environments/README.md) for details.

This notebook uses AVIRIS Indian Pine dataset for running. Take a look in [README](../../Datasets/IndianPines/README.md) for details.
  
- Tutorial: 
  - [Reading and Visualizing GeoTiff | Satellite Images with Python](https://towardsdatascience.com/reading-and-visualizing-geotiff-images-with-python-8dcca7a74510)
  - [Hyperspectral Image Analysis — Classification](https://towardsdatascience.com/hyperspectral-image-analysis-classification-c41f69ac447f)
  - [Hyperspectral Image Classification](https://www.intechopen.com/chapters/70188)

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [ ]:
import numpy as np
from scipy import io
import spectral

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_theme()

## 1. Load GeoTiff

In [ ]:
data_folder = "Data/"

Pure_TrSet = io.loadmat(data_folder + 'Pure_TrSet.mat')
Mixed_TrSet = io.loadmat(data_folder + 'Mixed_TrSet.mat')

TrLabel = io.loadmat(data_folder + 'TrLabel.mat')
TeLabel = io.loadmat(data_folder + 'TeLabel.mat')

Pure_TrSet, Mixed_TrSet = Pure_TrSet['Pure_TrSet'], Mixed_TrSet['Mixed_TrSet']
TrLabel, TeLabel = TrLabel['TrLabel'], TeLabel['TeLabel']

In [ ]:
print(Pure_TrSet.shape, Mixed_TrSet.shape)
print(TrLabel.shape, TeLabel.shape)

Mixed_TrSet = Mixed_TrSet.reshape(200, 200, Mixed_TrSet.shape[1])
print(Mixed_TrSet.shape)

In [ ]:
synthetic_data = io.loadmat("Data/synthetic_data_1.mat")
synthetic_data.keys()

In [ ]:
synthetic_data['X'].shape

In [ ]:
img = synthetic_data['X']
img = img.reshape(200, 200, img.shape[1])

# For no. of bands and resolution
(nrows, ncols, nbands) = img.shape
print(img.shape)
# print("\n".join(["Bands:\t"+str(nbands),"Rows:\t"+str(nrows),"Cols:\t"+str(ncols)]))

In [ ]:
hi_res = io.loadmat("hi_res.mat")
hi_res = hi_res["hi_res"]
print(hi_res.shape)

In [ ]:
def norm_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

norm_hi_res = norm_data(hi_res)

In [ ]:
bands=(30, 90, 150)

spectral.imshow(img, bands=bands, figsize=(6, 6) )
plt.grid(False) 
plt.axis('off')

spectral.imshow(Mixed_TrSet, bands=bands, figsize=(6, 6) )
plt.grid(False) 
plt.axis('off')

v = spectral.imshow(norm_hi_res, bands=bands, figsize=(6, 6) )
plt.grid(False) 
plt.axis('off')

In [ ]:
n_img = io.loadmat(r"abund.mat")
n_img = n_img['abund']
print(n_img.shape)

n_img = n_img.reshape(200, 200, 5)
print(n_img.shape)

In [ ]:
A_true = synthetic_data['A_true']
print(A_true.shape)

A_true = A_true.reshape(200, 200, 5)
print(A_true.shape)

In [ ]:
titles = ["#1 Roof", "#2 Asphalt", "#3 Soil", "#4 Water", "#5 Vegetation"]
fig, axes = plt.subplots(nrows=2, ncols=5, figsize = (30, 12) )

for i in range(n_img.shape[2]):
    axes[0][i].imshow(A_true[:,:,i], cmap='nipy_spectral')
    
    axes[0][i].grid(False) 
    axes[0][i].axis('off')
    axes[0][i].set_title(f"GT: {titles[i]}")
    
    axes[1][i].imshow(n_img[:,:,i], cmap='nipy_spectral')
    
    axes[1][i].grid(False) 
    axes[1][i].axis('off')
    axes[1][i].set_title(f"PD: {titles[i]}")

fig.tight_layout()

In [ ]:
from sklearn import metrics

rmse, mae = [], []
for i in range(n_img.shape[2]):
    rmse.append(metrics.mean_squared_error(A_true[:,:,i].flatten(), n_img[:,:,i].flatten()))
    mae.append(metrics.mean_absolute_error(A_true[:,:,i].flatten(), n_img[:,:,i].flatten()))

print("rmse:", sum(rmse)/len(rmse), rmse )
print("mae:", sum(mae)/len(mae), mae )

In [ ]:
# ", ".join( map( str, Mixed_TrSet[:,:,10].flatten() ) ) 
# ", ".join( map( str, hi_res[:,:,10].flatten() ) )

In [ ]:
rmse, mae = [], []
for i in range(hi_res.shape[2]):
    rmse.append(metrics.mean_squared_error(Mixed_TrSet[:,:,i].flatten(), hi_res[:,:,i].flatten()))
    mae.append(metrics.mean_absolute_error(Mixed_TrSet[:,:,i].flatten(), hi_res[:,:,i].flatten()))

print("rmse:", sum(rmse)/len(rmse), sorted(rmse, reverse=True) )
print("mae:", sum(mae)/len(mae), sorted(mae, reverse=True) )